In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install -q datasets
!pip install -q transforms
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import datasets
from datasets import load_dataset

In [ ]:
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
from torchvision import transforms as transforms
import torch

In [ ]:
# Define a class for the CocoCaptionDataset, which encapsulates the functionality for image captioning.
class CocoCaptionDataset:

    # Constructor initializes the model checkpoint and processor using Hugging Face's AutoProcessor.
    def __init__(self):
        # Define the Hugging Face model checkpoint for image captioning.
        self.model_ckpt = "HuggingFaceM4/idefics-9b-instruct"

        # Initialize the processor using the AutoProcessor with authentication token enabled.
        self.processor = AutoProcessor.from_pretrained(self.model_ckpt, use_auth_token=True)

    # Function to load and split the dataset into training and validation sets.
    def load_data(self):
        # Load the dataset using the "load_dataset" function from the datasets library.
        # The dataset is assumed to be located at the path "cat-state/mscoco-1st-caption".
        data = load_dataset("cat-state/mscoco-1st-caption")

        # Split the loaded dataset into training and testing sets.
        # The split ratio is set to allocate 0.02% of the data for testing.
        data = data["train"].train_test_split(test_size=0.0002)

        # Retrieve the training and validation sets after the split.
        train_data = data["train"]
        val_data = data["test"]

        # Return the training and validation sets for further processing or training.
        return train_data, val_data

    # Function to convert an image to RGB format if it is not already in that format.
    def img_convert_to_rgb(self, image):
        if image.mode == "RGB":
            return image
        image_rgba = image.convert("RGBA")
        background = Image.new("RGBA", image_rgba.size, (255, 255, 255))
        alpha_composite = Image.alpha_composite(background, image_rgba)
        alpha_composite = alpha_composite.convert("RGB")
        return alpha_composite

    # Function to transform the data for model input based on defined image processing steps.
    def transform_data(self, example):
        # Retrieve image size, mean, and standard deviation from the processor.
        img_size = self.processor.image_processor.image_size
        img_mean = self.processor.image_processor.image_mean
        img_std = self.processor.image_processor.image_std

        # Define a sequence of image transformation steps using torchvision.transforms.Compose.
        img_transform = transforms.Compose([
            self.img_convert_to_rgb,
            transforms.RandomResizedCrop((img_size, img_size), scale=(0.9, 1.0), interpolation=transforms.InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=img_mean, std=img_std),
        ])

        # Create prompts by combining image URL and captions in a specific format.
        prompts = []
        for i in range(len(example['caption'])):
            caption = example['caption'][i]
            prompts.append(
                [
                    example['url'][i],
                    f"Question: Explain the picture. Answer: {caption}</s>",
                ],
            )

        # Process prompts using the processor with the specified transformation and move to GPU.
        inputs = self.processor(prompts, transform=img_transform, return_tensors="pt").to("cuda")
        # Set 'labels' in inputs to be equal to 'input_ids'.
        inputs["labels"] = inputs["input_ids"]

        # Return the transformed inputs for model training or evaluation.
        return inputs

    # Function to generate training and validation datasets with the specified transformations.
    def gen_data(self):
        # Load the training and validation datasets.
        train_dataset, val_dataset = self.load_data()

        # Set the transformation function for both training and validation datasets.
        train_dataset.set_transform(self.transform_data)
        val_dataset.set_transform(self.transform_data)

        # Return the transformed training and validation datasets.
        return train_dataset, val_dataset

In [ ]:
# Import the previously defined CocoCaptionDataset class.
# from coco_caption_dataset import CocoCaptionDataset
# Define a class for the ImageCaptioningModel, which encapsulates the functionality for training and pushing the model to the Hugging Face Model Hub.
class ImageCaptioningModel:

    # Constructor initializes the CocoCaptionDataset, loads data, and sets the model checkpoint.
    def __init__(self):
        # Create an instance of CocoCaptionDataset to load and preprocess data.
        cococaptiondataset = CocoCaptionDataset()

        # Generate training and validation datasets.
        self.train_data, self.val_data = cococaptiondataset.gen_data()

        # Define the Hugging Face model checkpoint for image captioning.
        self.model_ckpt = "HuggingFaceM4/idefics-9b-instruct"

    # Function to load the vision-text2text model with quantization and specific configurations.
    def load_model(self):
        # Configure quantization settings using BitsAndBytesConfig.
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            llm_int8_skip_modules=["lm_head", "embed_tokens"],
        )

        # Load the IdeficsForVisionText2Text model with the specified quantization configuration.
        model = IdeficsForVisionText2Text.from_pretrained(self.model_ckpt, quantization_config=bnb_config, device_map="auto")

        # Return the loaded model.
        return model

    # Function to create a LoRA (LoraConfig) model using the provided model and configuration.
    def create_lora_model(self, model):
        # Configure LoRA (LoraConfig) settings.
        config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["q_proj", "k_proj", "v_proj"],
            lora_dropout=0.05,
            bias="none",
        )

        # Get the Peft model with LoRA configuration.
        lora_model = get_peft_model(model, config)

        # Print trainable parameters of the LoRA model.
        lora_model.print_trainable_parameters()

        # Return the created LoRA model.
        return lora_model

    # Function to set training arguments for the Trainer.
    def set_training_args(self):
        # Configure training arguments using TrainingArguments.
        # Define training arguments for the Trainer using the TrainingArguments class.

        # Specify the output directory where the trained model and logs will be saved.
        output_dir = "idefics-mscoco-captioner"

        # Set the learning rate for the optimizer during training.
        learning_rate = 2e-4

        # Enable mixed-precision training using 16-bit floating-point (fp16).
        fp16 = True

        # Set the number of training samples processed per batch on each device.
        per_device_train_batch_size = 2

        # Set the number of evaluation samples processed per batch on each device.
        per_device_eval_batch_size = 2

        # Accumulate gradients over this many training steps before performing a backward/update pass.
        gradient_accumulation_steps = 8

        # Disable pinning data into memory for DataLoader (dataloader_pin_memory).
        dataloader_pin_memory = False

        # Limit the total number of checkpoints saved during training.
        save_total_limit = 1

        # Define the strategy for evaluation during training (can be "steps", "epoch", or "no").
        evaluation_strategy = "steps"

        # Define the strategy for saving checkpoints during training (can be "steps", "epoch", or "no").
        save_strategy = "steps"

        # Save a checkpoint every specified number of training steps.
        save_steps = 50

        # Evaluate the model every specified number of training steps.
        eval_steps = 50

        # Log metrics every specified number of training steps.
        logging_steps = 50

        # Limit the total number of training steps during training.
        max_steps = 100

        # Keep columns that are not used during training and evaluation.
        remove_unused_columns = False

        # Load the best model at the end of training based on the evaluation metric.
        load_best_model_at_end = True

        # Specify the optimizer to be used during training (paged_adamw_8bit in this case).
        optim = "paged_adamw_8bit"

        # Specify the names of the labels in the dataset.
        label_names = ["labels"]

        # Create a TrainingArguments object with the specified configurations.
        training_args = TrainingArguments(
            output_dir=output_dir,
            learning_rate=learning_rate,
            fp16=fp16,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_eval_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            dataloader_pin_memory=dataloader_pin_memory,
            save_total_limit=save_total_limit,
            evaluation_strategy=evaluation_strategy,
            save_strategy=save_strategy,
            save_steps=save_steps,
            eval_steps=eval_steps,
            logging_steps=logging_steps,
            max_steps=max_steps,
            remove_unused_columns=remove_unused_columns,
            load_best_model_at_end=load_best_model_at_end,
            optim=optim,
            label_names=label_names
        )


        # Return the configured training arguments.
        return training_args

    # Function to train the model using Trainer and push the model to the Hugging Face Models Hub.
    def train_and_push_to_hub(self):
        # Load the model and create the LoRA model.
        img_cap_model = self.load_model()
        img_cap_model = self.create_lora_model(img_cap_model)

        # Configure Trainer with the loaded model, training arguments, and datasets.
        trainer = Trainer(
            model=img_cap_model,
            args=self.set_training_args(),
            train_dataset=self.train_data,
            eval_dataset=self.val_data
        )

        # Train the model using the configured Trainer.
        trainer.train()
        # drive.mount('/content/drive')
        # Push the trained model to the Hugging Face Model Hub.
        img_cap_model.push_to_hub("idefics-mscoco-captioner", private=False)
        # trainder.save_model("my_image_captioning_model")
        # trainer.save_model("/content/drive/MyDrive/my_image_captioning_model")

In [ ]:
if __name__ == "__main__":
    imagecaptioningmodel = ImageCaptioningModel()
    imagecaptioningmodel.train_and_push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/processing_auto.py:230: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You may have used the wrong order for inputs. `images` should be passed before `text`. The `images` and `text` inputs will be swapped. This behavior will be deprecated in transformers v4.47.


Step,Training Loss,Validation Loss
50,1.198200,0.559167
100,0.687900,0.547775


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/79.1M [00:00<?, ?B/s]

In [ ]:
# Function to perform inference using a given model, processor, and prompts.

# Parameters:
# - model: The pretrained language model for text generation.
# - processor: The processor associated with the model, responsible for tokenization and decoding.
# - prompts: List of input prompts for text generation.
# - max_new_tokens: Maximum number of new tokens to generate in the output (default is 50).
def check_inference(model, processor, prompts, max_new_tokens=50):
    # Get the tokenizer from the processor.
    tokenizer = processor.tokenizer

    # Define a list of bad words to be avoided in the generated output.
    bad_words = ["<image>", "<fake_token_around_image>"]

    # Tokenize the bad words and obtain their input_ids.
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    # Define the end-of-sequence token and obtain its input_id.
    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    # Tokenize the input prompts and convert them to PyTorch tensors.
    inputs = processor(prompts, return_tensors="pt").to("cuda")

    # Generate text using the model, specifying the end-of-sequence token, bad words, and maximum tokens.
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=True)

    # Decode the generated token IDs into text, skipping special tokens, and print the result.
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # print("-----------")
    # print(generated_text)
    return generated_text

In [ ]:
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            llm_int8_skip_modules=["lm_head", "embed_tokens"],
)

model = IdeficsForVisionText2Text.from_pretrained("HuggingFaceM4/idefics-9b-instruct", quantization_config=bnb_config, device_map="auto")

# Save the model to a directory in Colab (or directly to Google Drive if mounted)
model.save_pretrained("/content/drive/MyDrive/my_image_captioning_model")
# processor.save_pretrained("/content/drive/MyDrive/my_image_captioning_model")  # if processor is used


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
processor = AutoProcessor.from_pretrained("HuggingFaceM4/idefics-9b-instruct", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/processing_auto.py:230: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
from peft import PeftModel, PeftConfig
config = PeftConfig.from_pretrained("/content/idefics-mscoco-captioner")
model = PeftModel.from_pretrained(model,"/content/idefics-mscoco-captioner")

In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)

# Set ngrok auth token (replace with your actual token)
ngrok.set_auth_token("2pOx1uU30lU6JB4mdyDBYiwHoKL_Qt25okTkaFESV1Vqbq2Y")

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Home route serving HTML with inline JavaScript
@app.route("/")
def home():
    return """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Simple Flask App</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background: linear-gradient(135deg, #f5f7fa, #c3cfe2);
            color: #333;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            min-height: 100vh;
        }

        h1 {
            font-size: 2.5rem;
            color: #444;
            text-align: center;
            margin-bottom: 20px;
        }

        form {
            background: #fff;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            display: flex;
            flex-direction: column;
            align-items: center;
            gap: 10px;
            width: 100%;
            max-width: 400px;
        }

        label {
            font-size: 1.2rem;
            font-weight: bold;
            margin-bottom: 5px;
        }

        input[type="text"] {
            width: 100%;
            padding: 10px;
            font-size: 1rem;
            border: 2px solid #ccc;
            border-radius: 5px;
            outline: none;
            transition: border-color 0.3s;
        }

        input[type="text"]:focus {
            border-color: #007bff;
        }

        button {
            padding: 10px 20px;
            font-size: 1rem;
            color: #fff;
            background: #007bff;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s;
        }

        button:hover {
            background: #0056b3;
        }

        h3 {
            font-size: 1.5rem;
            margin-top: 20px;
            color: #555;
        }

        #outputBox {
            background: #fff;
            border: 1px solid #ccc;
            padding: 15px;
            border-radius: 5px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            font-size: 1rem;
            width: 100%;
            max-width: 400px;
            min-height: 50px;
            display: flex;
            align-items: center;
            justify-content: center;
        }
    </style>
    <script>
        async function handleSubmit(event) {
            event.preventDefault();
            const inputBox = document.getElementById('inputBox');
            const outputBox = document.getElementById('outputBox');
            const userInput = inputBox.value;

            const response = await fetch('/generate', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/x-www-form-urlencoded',
                },
                body: `user_input=${encodeURIComponent(userInput)}`,
            });

            const data = await response.json();
            outputBox.innerText = data.output;
        }
    </script>
</head>
<body>
    <h1>Image Caption Generator</h1>
    <form onsubmit="handleSubmit(event)">
        <label for="inputBox">Enter URL of image:</label>
        <input type="text" id="inputBox" name="user_input" required>
        <button type="generate">Submit</button>
    </form>
    <h3>Caption:</h3>
    <div id="outputBox"></div>
</body>
</html>
"""

# Route to process input and generate response
@app.route('/generate', methods=['POST'])
def generate_fnc():
    url = request.form['user_input']
    prompts = [
        url,
        "Question: Explain the picture. Answer:",
    ]
    output = check_inference(model, processor, prompts, max_new_tokens=50).split(':')[-1]
    return jsonify({"output": output})
    # user_input = request.form['user_input']
    # result = user_input + " hello"
    # return jsonify({"output": result})

# Run the Flask app
app.run(port=5000)


Public URL: NgrokTunnel: "https://26b0-34-142-135-88.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [27/Nov/2024 15:45:11] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/Nov/2024 15:45:12] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [27/Nov/2024 15:45:23] "POST /generate HTTP/1.1" 200 -
Token indices sequence length is longer than the specified maximum sequence length for this model (9082 > 2048). Running this sequence through the model will result in indexing errors
ERROR:__main__:Exception on /generate [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/

In [ ]:
!python -m pip install oracledb --upgrade --user

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.8 MB/s eta 0:00:00


In [ ]:
!pip install oracledb --upgrade --user

In [ ]:
!pip install oracledb

In [ ]:
!wget https://download.oracle.com/otn_software/nt/instantclient/2360000/instantclient-basic-windows.x64-23.6.0.24.10.zip

--2024-11-27 16:01:55--  https://download.oracle.com/otn_software/nt/instantclient/2360000/instantclient-basic-windows.x64-23.6.0.24.10.zip
Resolving download.oracle.com (download.oracle.com)... 23.208.168.100
Connecting to download.oracle.com (download.oracle.com)|23.208.168.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127144951 (121M) [application/zip]
Saving to: ‘instantclient-basic-windows.x64-23.6.0.24.10.zip’

instantclient-basic 100%[===================>] 121.25M   134MB/s    in 0.9s    

2024-11-27 16:01:56 (134 MB/s) - ‘instantclient-basic-windows.x64-23.6.0.24.10.zip’ saved [127144951/127144951]



In [ ]:
!unzip instantclient-basic-windows.x64-23.6.0.24.10.zip

Archive:  instantclient-basic-windows.x64-23.6.0.24.10.zip
  inflating: META-INF/MANIFEST.MF    
  inflating: META-INF/ORACLE_C.SF    
  inflating: META-INF/ORACLE_C.RSA   
   creating: instantclient_23_6/
  inflating: instantclient_23_6/adrci.exe  
  inflating: instantclient_23_6/adrci.sym  
  inflating: instantclient_23_6/BASIC_LICENSE  
  inflating: instantclient_23_6/BASIC_README  
  inflating: instantclient_23_6/extks.dll  
  inflating: instantclient_23_6/fips.dll  
  inflating: instantclient_23_6/genezi.exe  
  inflating: instantclient_23_6/genezi.sym  
  inflating: instantclient_23_6/legacy.dll  
   creating: instantclient_23_6/network/
   creating: instantclient_23_6/network/admin/
  inflating: instantclient_23_6/network/admin/README  
  inflating: instantclient_23_6/oci.dll  
  inflating: instantclient_23_6/oci.sym  
  inflating: instantclient_23_6/ocijdbc23.dll  
  inflating: instantclient_23_6/ocijdbc23.sym  
  inflating: instantclient_23_6/ociw32.dll  
  inflating: instantc

In [ ]:
import os
os.environ["LD_LIBRARY_PATH"] = "/content/instantclient_23_6"
os.environ["ORACLE_HOME"] = "/content/instantclient_23_6"
os.environ["PATH"] = f"/content/instantclient_23_6:{os.environ['PATH']}"

In [ ]:
!pip install --upgrade --force-reinstall oracledb

  Using cached oracledb-2.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
Using cached oracledb-2.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl (2.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.22
    Uninstalling pycparser-2.22:
      Successfully uninstalled pycparser-2.22
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: ora

In [ ]:
!pip list | grep oracledb

oracledb                           2.5.0


In [ ]:
import oracledb

In [ ]:
with oracledb.connect(user="sajal", password="password",host="localhost", port=1521, service_name="orclpdb") as connection:
    with connection.cursor() as cursor:
        sql = """select sysdate from dual"""
        for r in cursor.execute(sql):
            print(r)

OperationalError: DPY-6005: cannot connect to database (CONNECTION_ID=F9Oj3STwUTyS2oq+1nThVQ==).
[Errno 99] Cannot assign requested address

In [ ]:
public_url = ngrok.connect(1521, "tcp")
print("ngrok Public URL:", public_url)

PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: You must add a credit or debit card before you can use TCP endpoints on a free account. We require a valid card as a way to combat abuse and keep the internet a safe place. This card will NOT  be charged.\nAdd a card to your account here: https://dashboard.ngrok.com/settings#id-verification\r\n\r\nERR_NGROK_8013\r\n"}}
